In [ ]:
import numpy as np
import sys
import os
sys.path.insert(0, '/home/fbuonerba/codes/')
from mp_functions import upload_log_return, upload_factor_loadings, request_rates
from coinapi_v1 import CoinAPIv1
import datetime
from datetime import datetime, timedelta
import time
import calendar
import json
import csv

datapath='/home/databot/data/coinapi/'
homepath='/home/fbuonerba/'

ratespath=homepath+'exchange_rates_data/'
ohlcvpath=homepath + 'ohlcv_data/'

In [ ]:
rates=os.listdir(ratespath)

ohlcv=os.listdir(ohlcvpath)

In [ ]:
########TRANSFORM RATES############
D={}
counter=0
for x in rates:
    counter+=1
    if counter % 100000 ==0:
        print('golaso!')
    with open(ratespath + x) as infile:
        try:
            y=json.load(infile)
            if y!={} and y!='{}':
                ytime=y['time'][:10].replace('-','')
                if ytime not in D.keys():
                    D[ytime]=[]
                D[ytime].append(y)
        except:
            print('non existent data', x)
    
        

In [ ]:
days=list(D.keys())
names=['time', 'rate', 'asset_id_base', 'asset_id_quote']
for w in days:
    with open(datapath + w[:4]+'/'+w[4:6]+'/'+w[6:]+'/'+w+'.rates','w') as outfile:
        writer = csv.DictWriter(outfile, fieldnames=names)
        writer.writeheader()
        for symbol in D[w]:
            writer.writerow(symbol)

In [ ]:
###########TRANSFORM OHLCV#############
E={}
counter=0
for x in ohlcv:
    X=x.split('_')
    exchange=X[1]
    base=X[3]
    quote=X[4]
    if base=='SPOT':
        base=X[4]
        quote=X[2]
    counter+=1
    if counter % 50000 ==0:
        print(counter)
    with open(ohlcvpath + x) as infile:
        try:
            y=json.load(infile)
            if y!=[] and y!='[]':
                ytime=y['time_period_start'][:10].replace('-','')
                if ytime not in E.keys():
                    E[ytime]=[]
                y['exchange']=exchange
                y['asset_id_base']=base
                y['asset_id_quote']=quote
                if base=='SPOT' or quote=='SPOT' or exchange=='SPOT':
                    print(base, quote, exchange)
                E[ytime].append(y)
        except:
            pass

print(counter)            


In [ ]:
days=list(E.keys())
names=["time_close","time_open","price_close","price_open", 'asset_id_base','asset_id_quote','exchange',"trades_count","volume_traded", "price_high", "price_low","time_period_start","time_period_end"] 
for w in days:
    with open(datapath + w[:4]+'/'+w[4:6]+'/'+w[6:]+'/'+w+'.ohlcv','w') as outfile:
        writer = csv.DictWriter(outfile, fieldnames=names)
        writer.writeheader()
        for symbol in E[w]:
            writer.writerow(symbol)

In [ ]:
returnspath=homepath + 'log_returns_data/'

returns=os.listdir(returnspath)

In [ ]:
###############TRANSFORM RETURNS##############
F={}
counter=0
for x in returns:
    x_temp=x[:-4]
    X=x_temp.split('_')
    base=X[2]
    quote=X[3]
    time_begin=datetime.utcfromtimestamp(float(X[4])).strftime('%Y-%m-%dT%H:%M:%S')
    time_end=datetime.utcfromtimestamp(float(X[5])).strftime('%Y-%m-%dT%H:%M:%S')
    tim=datetime.utcfromtimestamp(float(X[5])).strftime('%Y%m%d')
    if tim not in F.keys():
        F[tim]=[]
    counter+=1
    if counter % 50000 ==0:
        print(counter)
    with open(returnspath + x) as infile:
        try:
            Y=json.load(infile)
            y={}
            y['time_start']=time_begin
            y['time_end']=time_end
            y['asset_id_base']=base
            y['asset_id_quote']=quote
            y['log_return']=Y
            F[tim].append(y)
        except:
            pass

print(counter)            


In [ ]:
days=list(F.keys())
names=['time_start', 'time_end', 'asset_id_base', 'asset_id_quote', 'log_return']
for w in days:
    try:
        with open(datapath + w[:4]+'/'+w[4:6]+'/'+w[6:]+'/'+w+'.log_returns','w') as outfile:
            writer = csv.DictWriter(outfile, fieldnames=names)
            writer.writeheader()
            for symbol in F[w]:
                writer.writerow(symbol)
    except:
        print(w, F[w])

In [ ]:
##########TRANSFORM CMC DAILY DATA################
G={}
cmcpath=homepath + 'cmc_data/'
cmc=os.listdir(cmcpath)
cmc1=[x for x in cmc if ('hist' not in x)]

for x in cmc1:
    date = datetime.utcfromtimestamp(float(x[-14:-4])).strftime('%Y%m%d')
    if date not in G.keys():
        G[date]=[]
    with open(cmcpath + x) as infile:
        j=json.load(infile)
    G[date].append(j)
keys=list(G[date][0].keys())


In [ ]:
days=G.keys()
for d in days:
    with open(datapath+d[:4]+'/'+d[4:6]+'/'+d[6:]+'/'+d+'.cmc_daily','w') as outfile:
        writer = csv.DictWriter(outfile, fieldnames=keys)
        writer.writeheader()
        for x in G[d]:
            try:
                writer.writerow(x)
            except:
                pass

In [ ]:
##########TRANSFORM CMC HISTORICAL DATA################
H={}

cmc2=[x for x in cmc if ('hist' in x)]
for x in cmc2:
    date = datetime.utcfromtimestamp(float(x[-14:-4])).strftime('%Y%m%d')
    if date not in H.keys():
        H[date]=[]
    with open(cmcpath + x) as infile:
        j=json.load(infile)
        for co in list(j.keys()):
            j[co]['symbol']=co
    H[date].append(j.values())
#keys1=list(H[date][0].keys())

In [ ]:
names=['symbol','supply','price', 'volume']
days=H.keys()
for d in days:
    with open(datapath+d[:4]+'/'+d[4:6]+'/'+d[6:]+'/'+d+'.cmc_historical','w') as outfile:
        writer = csv.DictWriter(outfile, fieldnames=names)
        writer.writeheader()
        for x in H[d][0]:
            try:
                writer.writerow(x)
            except:
                print(d, x)

In [ ]:
############COMPUTE DAILY VOLUMES###############
for root, dirs, files in os.walk(datapath, topdown=False):
    for randomfile in files:
        randomfile=os.path.join(root, randomfile)
        if 'ohlcv' in randomfile:
            volume={}
            with open(randomfile, 'r') as csvfile:
                sbam = csv.DictReader(csvfile)
                for row in sbam:
                    rowsym=row['asset_id_base']+'_'+row['asset_id_quote']
                    if 'SPOT' in rowsym:
                        print(randomfile)
                    if rowsym not in volume.keys():
                        volume[rowsym]={}
                        volume[rowsym]['time_period_start']=row['time_period_start']
                        volume[rowsym]['time_period_end']=row['time_period_end']
                        volume[rowsym]['asset_id_base']=row['asset_id_base']
                        volume[rowsym]['asset_id_quote']=row['asset_id_quote']
                        volume[rowsym]['volume_traded']=0
                    volume[rowsym]['volume_traded']+=float(row['volume_traded'])
            names=['time_period_start','time_period_end','asset_id_base','asset_id_quote','volume_traded']
            pairs=list(volume.keys())
            with open(randomfile[:-5]+'volumes','w') as outfile:
                #print(randomfile[:-5]+'volumes')
                writer = csv.DictWriter(outfile, fieldnames=names)
                writer.writeheader()
                for w in pairs:
                    writer.writerow(volume[w])